In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras. preprocessing.sequence import pad_sequences
print(tf.__version__)

from sklearn.model_selection import train_test_split

2.4.1


In [2]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import collections

In [5]:
test = pd.read_csv('C:/Users/mandy/OneDrive/Documents/GitHub/2021-Shopee-Code-League_Address-Elements-Extraction/test.csv')
train = pd.read_csv('C:/Users/mandy/OneDrive/Documents/GitHub/2021-Shopee-Code-League_Address-Elements-Extraction/train.csv')

print(train.head())
print(test.head())

   id                                        raw_address  \
0   0  jl kapuk timur delta sili iii lippo cika 11 a ...   
1   1                                 aye, jati sampurna   
2   2               setu siung 119 rt 5 1 13880 cipayung   
3   3                               toko dita, kertosono   
4   4                                      jl. orde baru   

                                  POI/street  
0  /jl kapuk timur delta sili iii lippo cika  
1                                          /  
2                                     /siung  
3                                 toko dita/  
4                             /jl. orde baru  
   id                                    raw_address
0   0          s. par 53 sidanegara 4 cilacap tengah
1   1          angg per, baloi indah kel. lubuk baja
2   2                          asma laun, mand imog,
3   3  ud agung rej, raya nga sri wedari karanganyar
4   4                     cut mutia, 35 baiturrahman


In [ ]:
print(train['raw_address'].sample(20))

In [ ]:
# creating regex to split raw_address into words
my_regex = '\s|,\s|,|\.|:'

# POI

## Splitting Data - POI

In [ ]:
raw_df_withPOI = train[train['POI/street'].str.contains('.+(?=/)')==True]
raw_df_withPOI['POI/street'] = raw_df_withPOI['POI/street'].str.extract(r'(.+)(?=/)')
raw_df_withPOI.head()

In [ ]:
# turning raw_address and POI/street columns into lists of strings; discarding id and index column
list_of_addresses_withPOI = [*raw_df_withPOI['raw_address']]
print(list_of_addresses_withPOI[:5])

list_of_POI = [*raw_df_withPOI['POI/street']]

## Creating labels_binary_POI

In [ ]:
max_length_address = 25   #len(max(raw_df_withPOI['raw_address']))
max_length_POI = 10       #len(max(raw_df_withPOI['POI/street']))

In [ ]:
dictionary_POI = {}

In [ ]:
# binary labels for training set
number_of_binary_labels = len(list_of_addresses_withPOI)
print(len(list_of_addresses_withPOI))
labels_binary_POI = [[]]*number_of_binary_labels


# for each word in 'raw_address', match it to words in 'POI/street' column.
# if there's a match, value 1; if no match, value 0
for i in range(number_of_binary_labels):
  label_binary = [0 for _ in range(max_length_address)]
  words = re.split(my_regex, list_of_POI[i])
  address = re.split(my_regex, list_of_addresses_withPOI[i])

  for word in words:
    for j, ad in enumerate(address):
      if word.startswith(ad):
        if (ad != word) & ((word in dictionary_POI.values())==False):
          dictionary_POI[ad] = word
        if j<max_length_address:
          label_binary[j] = 1
          break

  labels_binary_POI[i] = label_binary
 
labels_binary_POI = np.array(labels_binary_POI)
print(labels_binary_POI[:5])
print(dict(list(dictionary_POI.items())[0:5]))
print(len(dictionary_POI))

In [ ]:
print(len(dictionary_POI))

def filter_dict(mydict):
    filter_dict_num = {k: mydict[k] for k in sorted(mydict.keys()) if not any(map(str.isdigit, k))}
    filter_dict_len = {k: filter_dict_num[k] for k in sorted(filter_dict_num.keys()) if len(k)>0}
    return filter_dict_len

filtered_dict_POI = filter_dict(dictionary_POI)
print(len(filtered_dict_POI))

## Tokenizer and Padding - POI

In [ ]:
# # plotting distribution of length of raw_addresses and of POI
# length_of_addresses = [len(re.split('\s|, |\.|:', address)) for address in train_addresses_withPOI]
# print(length_of_addresses[:5])
# print(max(length_of_addresses))
# print(len(train_addresses_withPOI))

# A = collections.Counter(length_of_addresses)
# plt.bar(A.keys(), A.values())
# plt.show()

# length_of_POI = [len(re.split('\s|, |\.|:', POI)) for POI in train_labels_POI]
# print(length_of_POI[:5])
# print(max(length_of_POI))

# P = collections.Counter(length_of_POI)
# plt.bar(P.keys(), P.values())
# plt.show()

In [ ]:
# vocab_size = 200000
embedding_dim = 64
trunc_type = 'post'
padding = 'post'
oov_tok="<OOV>"

print(type(list_of_addresses_withPOI))
sentences_to_tokenize = list_of_addresses_withPOI.copy()

print(type(sentences_to_tokenize))

print(len(list_of_addresses_withPOI))

sentences_to_tokenize.append(list_of_POI)
print(len(list_of_addresses_withPOI))
print(sentences_to_tokenize[:5])


# sentences_to_tokenize = train_addresses.append(train_POI_sentences)
# print(sentences_to_tokenize[:5])
# for row in train_labels_POI:
#   train_addresses.append(row)
# print(sentences_to_tokenize[-5:])


# for row in test_labels_POI:
#   test_sentences.append(row)
# print(test_sentences[-5:])

In [ ]:
if len(list_of_addresses_withPOI) != len(list_of_POI):
  print(len(list_of_addresses_withPOI))
  print(len(list_of_POI))

In [ ]:
# Tokenizing train_addresses and train_POI

tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences_to_tokenize)

word_index = tokenizer.word_index
print(len(word_index))
total_words = len(word_index)+1

def tokenize_and_pad(data):
  data_sequences = tokenizer.texts_to_sequences(data)
  data_padded = pad_sequences(data_sequences, maxlen=max_length_address, truncating=trunc_type, padding=padding)
  return data_padded

#Padding addresses
padded = tokenize_and_pad(list_of_addresses_withPOI)
print(padded[0])
print(padded.shape)

In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(list_of_addresses_withPOI[1])

# Subword Tokenization
too advanced, skipping

In [ ]:
# #!pip install -q tensorflow_text_nightly
# !pip install -q tf-nightly
# !pip install tokenizers -q

# from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab
# from tokenizers import BertWordPieceTokenizer

In [ ]:
# # Initialize an empty BERT tokenizer
# tokenizer = BertWordPieceTokenizer(
#   clean_text=False,
#   handle_chinese_chars=False,
#   strip_accents=False,
#   lowercase=True,
# )

# # prepare text files to train vocab on them
# test_string = ['/content/test.txt']
# #[sentences_to_tokenize]
# #

# # train BERT tokenizer
# tokenizer.train(
#   test_string,
#   vocab_size=20,
#   min_frequency=2,
#   show_progress=True,
#   special_tokens=['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]'],
#   limit_alphabet=1000,
#   wordpieces_prefix="##"
# )

# # save the vocab
# tokenizer.save('/content/bert-vocab.txt')

# # create a BERT tokenizer with trained vocab
# vocab = '/content/bert-vocab.txt'
# tokenizer = BertWordPieceTokenizer(vocab)

# # test the tokenizer with some text
# encoded = tokenizer.encode('...')
# print(encoded.tokens)

In [ ]:
# #bert_tokenizer_params=dict(lower_case=True)
# reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"]

# bert_vocab_args = dict(
#     # The target vocabulary size
#     vocab_size = 97192,
#     # Reserved tokens that must be included in the vocabulary
#     reserved_tokens=reserved_tokens,
#     # Arguments for `text.BertTokenizer`
#     bert_tokenizer_params=dict(lower_case=True),
#     # Arguments for `wordpiece_vocab.wordpiece_tokenizer_learner_lib.learn`
#     learn_params={},
# )

# vocab = bert_vocab.bert_vocab_from_dataset(
#     sentences_to_tokenize, 
#     **bert_vocab_args
# )


In [ ]:
# Tokenize corpus (all words AI should learn) without vocab_size limit 

# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(sentences_to_tokenize)

# word_index = tokenizer.word_index
# print(len(word_index))


## Train Test Split - POI

In [ ]:
# train_test_split to get training and testing datasets
train_addresses_withPOI, test_addresses_withPOI, train_labels_POI, test_labels_POI = train_test_split(
padded, labels_binary_POI, test_size=0.20, random_state=42)

# print(len(padded))
# print(len(train_labels_POI))

## Model building - POI

In [ ]:
# basic model - no Bidirectional, LSTM, etc.
model_POI = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_words, embedding_dim, input_length=max_length_address),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(max_length_address, activation='sigmoid')
])
model_POI.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model_POI.summary()

In [ ]:
if train_padded.shape[0] != labels_binary_POI.shape[0]:
  print('Training addresses and labels have different number of rows')

In [ ]:
earlyStopping = EarlyStopping(monitor = 'val_acc', patience = 5)
modelCheckpoint = ModelCheckpoint('best_model.hdf5', save_best_only = True)

num_epochs = 3

history = model_POI.fit(train_addresses_withPOI, train_labels_POI, epochs=num_epochs, validation_data=(test_addresses_withPOI, test_labels_POI), callbacks=[earlyStopping, modelCheckpoint])

In [ ]:
def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

plot_graphs(history, 'accuracy')
plot_graphs(history, 'loss')

## model_POI.predict

In [ ]:
print(type(test))
live_padded = tokenize_and_pad(test['raw_address'])
print(live_padded.shape)

# live_to_predict = test_padded[:5]
# live_prediction_POI = model_POI.predict(live_to_predict)

def predict_and_convert_to_words(live_to_predict, raw_address, threshold, dictionary, model):
  live_prediction = model.predict(live_to_predict)
  number_of_live_labels = len(live_to_predict)
  # print(number_of_live_labels)

  live_labels = [[]]*number_of_live_labels
  
  # decode live_labels to regular words
  for i in range(number_of_live_labels):
    live_label = ''
    address = re.split(my_regex, raw_address.iloc[i])
  
    for id, j in enumerate(live_prediction[i]):
      if (j > threshold) & (id < len(address)):
        if address[id] in dictionary:
          # print(id, j, address[id])
          new_string = dictionary.get(address[id])
          live_label += new_string
          live_label += ' '
        else:
          # print(id, j, address[id])
          live_label += address[id]
          live_label += ' '
    # print(live_label_POI)
    live_label = live_label.rstrip()
    live_labels[i] = live_label
  return live_labels

live_labels_POI = predict_and_convert_to_words(live_padded, test['raw_address'], threshold=0.5, filtered_dict_POI, model_POI)
print(live_labels_POI[:5])
print(test['raw_address'][:5])

# if want to run test, will get error as train_addresses_withPOI is not a DataFrame
# test = predict_and_convert_to_words(test_padded[:5], train_addresses_withPOI, filtered_dict_POI, model_POI)
# print(test)

# Street (ST)

## Splitting Data - ST

In [ ]:
raw_df_withST = train[train['POI/street'].str.contains('(?<=/).+')==True]
raw_df_withST['POI/street'] = raw_df_withST['POI/street'].str.extract(r'(?<=/)(.+)')
raw_df_withST.head()

In [ ]:
# turning raw_address and POI/street columns into lists of strings; discarding id and index column
list_of_addresses_withST = [*raw_df_withST['raw_address']]
print(list_of_addresses_withST[:5])

list_of_ST = [*raw_df_withST['POI/street']]

## Creating labels_binary_ST

In [ ]:
# plotting distribution of length of raw_addresses and of POI
length_of_addresses_ST = [len(re.split('\s|, |\.|:', address)) for address in list_of_addresses_withST]
print(length_of_addresses_ST[:5])
print(max(length_of_addresses_ST))

A = collections.Counter(length_of_addresses_ST)
plt.bar(A.keys(), A.values())
plt.show()

length_of_ST = [len(re.split('\s|, |\.|:', ST)) for ST in list_of_ST]
print(length_of_ST[:5])
print(max(length_of_ST))

P = collections.Counter(length_of_ST)
plt.bar(P.keys(), P.values())
plt.show()

In [ ]:
max_length_address = 25   #len(max(raw_df_withST['raw_address']))
max_length_ST = 10       #len(max(raw_df_withST['POI/street']))

In [ ]:
dictionary_ST = {}

In [ ]:
# binary labels for training set
number_of_binary_labels = len(list_of_addresses_withST)
print(len(list_of_addresses_withST))
labels_binary_ST = [[]]*number_of_binary_labels


# for each word in 'raw_address', match it to words in 'POI/street' column.
# if there's a match, value 1; if no match, value 0
for i in range(number_of_binary_labels):
  label_binary = [0 for _ in range(max_length_address)]
  words = re.split(my_regex, list_of_ST[i])
  address = re.split(my_regex, list_of_addresses_withST[i])

  for word in words:
    for j, ad in enumerate(address):
      if word.startswith(ad):
        if (ad != word) & ((word in dictionary_ST.values())==False):
          dictionary_ST[ad] = word
        if j<max_length_address:
          label_binary[j] = 1
          break

  labels_binary_ST[i] = label_binary
 
labels_binary_ST = np.array(labels_binary_ST)
print(labels_binary_ST[:5])
print(list_of_addresses_withST[:5])
print(list_of_ST[:5])

In [ ]:
print(len(dictionary_ST))

# def filter_dict(mydict):
#     filter_dict_num = {k: mydict[k] for k in sorted(mydict.keys()) if not any(map(str.isdigit, k))}
#     filter_dict_len = {k: filter_dict_num[k] for k in sorted(filter_dict_num.keys()) if len(k)>0}
#     return filter_dict_len

filtered_dict_ST = filter_dict(dictionary_ST)
print(len(filtered_dict_ST))

## Tokenizer and Padding - ST

In [ ]:
embedding_dim = 64
trunc_type = 'post'
padding = 'post'
oov_tok="<OOV>"

print(type(list_of_addresses_withST))
sentences_to_tokenize = list_of_addresses_withST.copy()

print(type(sentences_to_tokenize))
# train_ST_sentences = []

print(len(list_of_addresses_withST))

sentences_to_tokenize.append(list_of_ST)
print(len(list_of_addresses_withST))
print(sentences_to_tokenize[:5])


# sentences_to_tokenize = train_addresses.append(train_ST_sentences)
# print(sentences_to_tokenize[:5])
# for row in list_of_ST:
#   train_addresses.append(row)
# print(sentences_to_tokenize[-5:])


# for row in test_labels_ST:
#   test_sentences.append(row)
# print(test_sentences[-5:])

In [ ]:
if len(list_of_addresses_withST) != len(list_of_ST):
  print(len(list_of_addresses_withPOI))
  print(len(list_of_POI))

In [ ]:
# Tokenizing train_addresses and train_ST

tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences_to_tokenize)

word_index = tokenizer.word_index
print(len(word_index))
total_words = len(word_index)+1

#Padding train_addresses, train_ST, test_addresses
padded = tokenize_and_pad(list_of_addresses_withST)
print(padded[0])
print(padded.shape)

In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(train_padded[1]))
print(list_of_addresses_withST[1])

## Train Test Split - ST

In [ ]:
# train_test_split to get training and testing datasets
train_addresses_withST, test_addresses_withST, train_labels_ST, test_labels_ST = train_test_split(
padded, labels_binary_ST, test_size=0.20, random_state=42)

print(len(train_addresses_withST))
print(len(train_labels_ST))

## Model building - ST

In [ ]:
# basic model - no Bidirectional, LSTM, etc.
model_ST = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_words, embedding_dim, input_length=max_length_address),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(max_length_address, activation='sigmoid')
])
model_ST.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model_ST.summary()

In [ ]:
if train_padded.shape[0] != labels_binary_ST.shape[0]:
  print('Training addresses and labels have different number of rows')

In [ ]:
num_epochs = 10

history = model_ST.fit(train_addresses_withST, train_labels_ST, epochs=num_epochs, validation_data=(test_addresses_withST, test_labels_ST), callbacks=[earlyStopping, modelCheckpoint])

In [ ]:
def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

plot_graphs(history, 'accuracy')
plot_graphs(history, 'loss')

## model_ST.predict

In [ ]:
live_padded_ST = tokenize_and_pad(test['raw_address'])
print(live_padded_ST.shape)

In [ ]:
live_labels_ST = predict_and_convert_to_words(live_padded_ST, test['raw_address'], threshold=0.5, filtered_dict_ST, model_ST)
print(live_labels_ST[:5])
print(test['raw_address'][:5])

# Combine results of POI and ST - POI/Street column

In [ ]:
print('live_labels_ST is type: ' + str(type(live_labels_ST)))
print('live_labels_POI is type: ' + str(type(live_labels_POI)))

# check if live_labels_ST and _POI have same number of elements
if len(live_labels_ST) != len(live_labels_POI):
  print('live_labels_ST and _POI have different number of elements')

In [ ]:
test['POI'] = live_labels_POI
test['Street'] = live_labels_ST

print(test.head())

test['POI/Street'] = test.POI.cat(test.Street, sep='/')
print(test.head())

submission = test[['id', 'POI/Street']]
print(submission)

submission.to_csv('submission.csv', index=False)